Things to change
1. SQL: Table name bigquery:`bigquery-public-data.epa_historical_air_quality.co_daily_summary`

2. CSV for 2021/2020 date :                      
    `df_co_2020 = pd.read_csv('daily_42101_2020.csv')`            
    `df_co_2021 = pd.read_csv('daily_42101_2021.csv')`

3. final name for CSV : `final.to_csv("co_daily.csv")`

Revisit: Interpolation technique for missing time series data

Pending:
1. Merge all datasets wither via a inner join or left join with master unique combo-> then remove rows where no data for all pollutants
2. AQI Claculator func: Given a pollutant convert its pollutant concentration to its AQI VALUE
3. Find the max AQI and corresponding Polutant for that county/state/date


In [2]:
import pandas as pd
#pip install --upgrade google-api-python-client
#pip install pip install google-cloud-bigquery
from google.cloud import bigquery
# pip install -e git+https://github.com/SohierDane/BigQuery_Helper#egg=bq_helper
from  bq_helper import BigQueryHelper
import os

Abhishek: ask what the above code does

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="//Users/abhishekgawande/Downloads/OneDrive_4_11-16-2021/fall-21-329000-5fc2cb5ab9a1.json"


bq_assist=BigQueryHelper("bigquery-public-data","epa_historical_air_quality")



In [4]:
%load_ext google.cloud.bigquery

In [5]:
# %%bigquery

# select date_local,state_code,county_code,count(*) as sites, max(mx-mn) as max_dev,  min(mx-mn) as min_dev
# from

# (SELECT
#    date_local,state_code,county_code,site_num,max(first_max_value)as mx, min(first_max_value) as mn
#         FROM
#         `bigquery-public-data.epa_historical_air_quality.co_daily_summary`
#  where date_local>='2010-01-01' and aqi >=0

# GROUP BY 1,2,3,4 )a
# group by 1,2,3
# order by max_dev desc
# LIMIT 100

In [6]:
query="""
   SELECT cnt, count(*) as cnt2
   FROM (SELECT
        date_local,state_code,county_code,count(*) as cnt
        FROM
        `bigquery-public-data.epa_historical_air_quality.co_daily_summary`
        where date_local>='2010-01-01' and aqi >=0
        GROUP BY 1,2,3
      )a
      GROUP BY cnt
    ORDER BY cnt DESC
    """


bq_df=bq_assist.query_to_pandas(query)
print(bq_df.head())
#bq_df.to_csv('file1.csv')

   cnt  cnt2
0   16   210
1   15  1165
2   14  1966
3   13  1283
4   12   375


In [7]:
query="""
 select sample_duration, case when aqi>=0 then 1 else 0 end as aqi_filter, count(*) from 
        `bigquery-public-data.epa_historical_air_quality.co_daily_summary`
        where date_local>='2010-01-01' ---AND sample_duration="8-HR RUN AVG END HOUR"
        group by 1,2
        order by 1,2
        
    """


bq_df=bq_assist.query_to_pandas(query)
print(bq_df.head())
#bq_df.to_csv('file1.csv')

##### USE FILTER sample_duration="8-HR RUN AVG END HOUR" or aqi>=0

         sample_duration  aqi_filter      f0_
0                 1 HOUR           0  1059904
1  8-HR RUN AVG END HOUR           1  1059879


## CO Daily Summary : co_daily_df

In [8]:
# Abhishek: 17 Nov 2021

bq_assist.list_tables()


['air_quality_annual_summary',
 'co_daily_summary',
 'co_hourly_summary',
 'hap_daily_summary',
 'hap_hourly_summary',
 'lead_daily_summary',
 'no2_daily_summary',
 'no2_hourly_summary',
 'nonoxnoy_daily_summary',
 'nonoxnoy_hourly_summary',
 'o3_daily_summary',
 'o3_hourly_summary',
 'pm10_daily_summary',
 'pm10_hourly_summary',
 'pm25_frm_daily_summary',
 'pm25_frm_hourly_summary',
 'pm25_nonfrm_daily_summary',
 'pm25_nonfrm_hourly_summary',
 'pm25_speciation_daily_summary',
 'pm25_speciation_hourly_summary',
 'pressure_daily_summary',
 'pressure_hourly_summary',
 'rh_and_dp_daily_summary',
 'rh_and_dp_hourly_summary',
 'so2_daily_summary',
 'so2_hourly_summary',
 'temperature_daily_summary',
 'temperature_hourly_summary',
 'voc_daily_summary',
 'voc_hourly_summary',
 'wind_daily_summary',
 'wind_hourly_summary']

In [9]:
# Abhishek: 17 Nov 2021

query="""
   SELECT * from `bigquery-public-data.epa_historical_air_quality.pressure_daily_summary`
   LIMIT 5
    """

bq_df=bq_assist.query_to_pandas(query)
# print(bq_df.head())
print(bq_df.columns)

Index(['state_code', 'county_code', 'site_num', 'parameter_code', 'poc',
       'latitude', 'longitude', 'datum', 'parameter_name', 'sample_duration',
       'pollutant_standard', 'date_local', 'units_of_measure', 'event_type',
       'observation_count', 'observation_percent', 'arithmetic_mean',
       'first_max_value', 'first_max_hour', 'aqi', 'method_code',
       'method_name', 'local_site_name', 'address', 'state_name',
       'county_name', 'city_name', 'cbsa_name', 'date_of_last_change'],
      dtype='object')


In [10]:
# Abhishek: 17 Nov 2021

query="""
   SELECT date_local from `bigquery-public-data.epa_historical_air_quality.co_daily_summary`
   LIMIT 1
    """

bq_df=bq_assist.query_to_pandas(query)
print(bq_df.dtypes)
print(bq_df.date_local.unique())

date_local    object
dtype: object
[datetime.date(2006, 2, 11)]


In [11]:
query="""



select date_local,
         cast(state_code as int) as state_code ,
         state_name,
         cast(county_code as int) as county_code,
         county_name,
      ---   city_name,
        --- cbsa_name,
         parameter_name,
         units_of_measure,
         max(AQI) as aqi_label,
         max(first_max_value) as polutant_level
         from `bigquery-public-data.epa_historical_air_quality.co_daily_summary`
         where date_local>='2010-01-01' AND aqi>=0
        group by 1,2,3,4,5,6,7
        order by 1,2,3,4,5,6,7
        
    """

co_daily_df=bq_assist.query_to_pandas(query)
print(co_daily_df.head())
co_daily_df.to_csv('file1.csv')


   date_local  state_code state_name  county_code            county_name  \
0  2010-01-01           1    Alabama           73              Jefferson   
1  2010-01-01           2     Alaska           20             Anchorage    
2  2010-01-01           2     Alaska           90  Fairbanks North Star    
3  2010-01-01           4    Arizona           13               Maricopa   
4  2010-01-01           4    Arizona           19                   Pima   

    parameter_name   units_of_measure  aqi_label  polutant_level  
0  Carbon monoxide  Parts per million          6             0.5  
1  Carbon monoxide  Parts per million         24             2.1  
2  Carbon monoxide  Parts per million         20             1.8  
3  Carbon monoxide  Parts per million         35             3.1  
4  Carbon monoxide  Parts per million          9             0.8  


In [12]:
# co_daily_df['date_local'] = pd.to_datetime(co_daily_df['date_local'], format='%Y-%m-%d')

# co_daily_df["yy"]=co_daily_df['date_local'].dt.year
# co_daily_df[co_daily_df["yy"]==2019].groupby(["yy","state_code","state_name","county_code"]).count().sort_values("polutant_level",ascending=False)

In [13]:
df_co_2020 = pd.read_csv('daily_42101_2020.csv')
df_co_2021 = pd.read_csv('daily_42101_2021.csv')
print(df_co_2020[df_co_2020["AQI"]>=0].shape[0])
print(df_co_2021[df_co_2020["AQI"]>=0].shape[0])
result_CO = df_co_2021[df_co_2020["AQI"]>=0].append(df_co_2020[df_co_2020["AQI"]>=0], sort=False)
print(result_CO.shape[0])
result_CO.head()
#list(result_CO.columns)
result_CO= result_CO[
[ 'Date Local',
'State Code', 
'State Name',
'County Name',
# 'City Name',
# 'CBSA Name',
'County Code',
'Parameter Name',
'Units of Measure',
'1st Max Value',
'AQI']]




result_CO_grouped=result_CO.groupby([ 
'Date Local',
'State Code', 
'State Name',
'County Name',
# 'City Name',
# 'CBSA Name',
'County Code',
'Parameter Name',
'Units of Measure'
]).agg({ 'AQI' :'max','1st Max Value':'max'}).reset_index().sort_values(
by=['Date Local',
'State Code', 
'State Name',
'County Name',
# 'City Name',
# 'CBSA Name',
'County Code',
'Parameter Name',
'Units of Measure'])
result_CO_grouped.head()
result_CO_grouped.shape
result_CO_grouped.columns=['date_local', 'state_code', 'state_name', 'county_name',
#        'city_name', 'cbsa_name', 
        'county_code','parameter_name', 'units_of_measure',
       'aqi_label', 'polutant_level']


89385
7351
96736


<ipython-input-13-8c9f0f4cc559>:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(df_co_2021[df_co_2020["AQI"]>=0].shape[0])
<ipython-input-13-8c9f0f4cc559>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result_CO = df_co_2021[df_co_2020["AQI"]>=0].append(df_co_2020[df_co_2020["AQI"]>=0], sort=False)


In [14]:
result_CO_grouped.head()

,date_local,state_code,state_name,county_name,county_code,parameter_name,units_of_measure,aqi_label,polutant_level
0,2020-01-01,1,Alabama,Jefferson,73,Carbon monoxide,Parts per million,7.0,0.6
1,2020-01-01,2,Alaska,Anchorage,20,Carbon monoxide,Parts per million,11.0,1.0
2,2020-01-01,4,Arizona,Maricopa,13,Carbon monoxide,Parts per million,34.0,3.0
3,2020-01-01,4,Arizona,Pima,19,Carbon monoxide,Parts per million,6.0,0.5
4,2020-01-01,5,Arkansas,Pulaski,119,Carbon monoxide,Parts per million,9.0,0.8


In [15]:
co_daily_df.head()

,date_local,state_code,state_name,county_code,county_name,parameter_name,units_of_measure,aqi_label,polutant_level
0,2010-01-01,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,6,0.5
1,2010-01-01,2,Alaska,20,Anchorage,Carbon monoxide,Parts per million,24,2.1
2,2010-01-01,2,Alaska,90,Fairbanks North Star,Carbon monoxide,Parts per million,20,1.8
3,2010-01-01,4,Arizona,13,Maricopa,Carbon monoxide,Parts per million,35,3.1
4,2010-01-01,4,Arizona,19,Pima,Carbon monoxide,Parts per million,9,0.8


In [16]:
merged_CO_Daily=co_daily_df.append(result_CO_grouped, sort=False)

In [17]:
merged_CO_Daily.head()

,date_local,state_code,state_name,county_code,county_name,parameter_name,units_of_measure,aqi_label,polutant_level
0,2010-01-01,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,6.0,0.5
1,2010-01-01,2,Alaska,20,Anchorage,Carbon monoxide,Parts per million,24.0,2.1
2,2010-01-01,2,Alaska,90,Fairbanks North Star,Carbon monoxide,Parts per million,20.0,1.8
3,2010-01-01,4,Arizona,13,Maricopa,Carbon monoxide,Parts per million,35.0,3.1
4,2010-01-01,4,Arizona,19,Pima,Carbon monoxide,Parts per million,9.0,0.8


In [18]:
merged_CO_Daily
merged_CO_Daily['date_local'] = pd.to_datetime(merged_CO_Daily['date_local'], format='%Y-%m-%d')
merged_CO_Daily.dtypes

date_local          datetime64[ns]
state_code                   int64
state_name                  object
county_code                  int64
county_name                 object
parameter_name              object
units_of_measure            object
aqi_label                  float64
polutant_level             float64
dtype: object

In [19]:


from datetime import datetime,date

merged_CO_Daily['Year'] = merged_CO_Daily['date_local'].dt.year

county_state_check=merged_CO_Daily[merged_CO_Daily["Year"]>=2019].groupby(["state_code","county_code"]).agg({"aqi_label":"count"}).sort_values("aqi_label", ascending=False)



In [20]:

# county_state_check=merged_CO_Daily[merged_CO_Daily["Year"]>=2019].groupby(["Year","state_code","county_code"]).agg({"aqi_label":"count"}).sort_values(["state_code","county_code","Year"])




In [21]:
# merged_CO_Daily.dtypes

In [22]:
county_state_check

,,aqi_label
state_code,county_code,
40,109,852
23,9,851
42,3,822
36,81,822
31,55,822
...,...,...
25,13,159
53,67,135
8,77,134


In [23]:
print(county_state_check.count()[0])
print(county_state_check[county_state_check["aqi_label"]>700].count()[0])

171
122


In [24]:
county_state_check_filtered=county_state_check[county_state_check["aqi_label"]>700].reset_index()
county_state_check_filtered=county_state_check_filtered[["state_code","county_code"]]
county_state_check_filtered

,state_code,county_code
0,40,109
1,23,9
2,42,3
3,36,81
4,31,55
...,...,...
117,12,31
118,24,5
119,17,31
120,6,25


In [25]:


merged_CO_Daily_filtered=merged_CO_Daily.merge(county_state_check_filtered, left_on=["state_code","county_code"], right_on=["state_code","county_code"])

merged_CO_Daily_filtered

,date_local,state_code,state_name,county_code,county_name,parameter_name,units_of_measure,aqi_label,polutant_level,Year
0,2010-01-01,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,6.0,0.5,2010
1,2010-01-02,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,6.0,0.5,2010
2,2010-01-03,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,6.0,0.5,2010
3,2010-01-04,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,8.0,0.7,2010
4,2010-01-05,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,8.0,0.7,2010
...,...,...,...,...,...,...,...,...,...,...
459351,2020-12-27,56,Wyoming,9,Converse,Carbon monoxide,Parts per million,1.0,0.1,2020
459352,2020-12-28,56,Wyoming,9,Converse,Carbon monoxide,Parts per million,1.0,0.1,2020
459353,2020-12-29,56,Wyoming,9,Converse,Carbon monoxide,Parts per million,1.0,0.1,2020
459354,2020-12-30,56,Wyoming,9,Converse,Carbon monoxide,Parts per million,1.0,0.1,2020


In [26]:
county_state_check2=merged_CO_Daily_filtered.groupby(["state_code","county_code"]).agg({"aqi_label":"count"}).sort_values("aqi_label", ascending=False)

county_state_check_filtered_v2=county_state_check2[county_state_check2["aqi_label"]>3650].reset_index()
county_state_check_filtered_v2=county_state_check_filtered_v2[["state_code","county_code"]]
county_state_check_filtered_v2

merged_CO_Daily_filtered_v2=merged_CO_Daily.merge(county_state_check_filtered_v2, left_on=["state_code","county_code"], right_on=["state_code","county_code"])




In [27]:
# print(county_state_check2.count()[0])
# print(county_state_check2[county_state_check2["aqi_label"]>3650].count()[0])

In [28]:
# 461/3650

In [29]:
merged_CO_Daily_filtered_v2

,date_local,state_code,state_name,county_code,county_name,parameter_name,units_of_measure,aqi_label,polutant_level,Year
0,2010-01-01,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,6.0,0.5,2010
1,2010-01-02,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,6.0,0.5,2010
2,2010-01-03,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,6.0,0.5,2010
3,2010-01-04,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,8.0,0.7,2010
4,2010-01-05,1,Alabama,73,Jefferson,Carbon monoxide,Parts per million,8.0,0.7,2010
...,...,...,...,...,...,...,...,...,...,...
398153,2020-12-27,55,Wisconsin,27,Dodge,Carbon monoxide,Parts per million,2.0,0.2,2020
398154,2020-12-28,55,Wisconsin,27,Dodge,Carbon monoxide,Parts per million,2.0,0.2,2020
398155,2020-12-29,55,Wisconsin,27,Dodge,Carbon monoxide,Parts per million,2.0,0.2,2020
398156,2020-12-30,55,Wisconsin,27,Dodge,Carbon monoxide,Parts per million,2.0,0.2,2020


In [30]:
100*4153

415300

In [31]:
398158/415300

0.9587238141102817

In [32]:
print(3650/4153)

print(17342+398158)

0.8788827353720202
415500


In [33]:
rng = pd.date_range('2010-01-01','2021-05-17',  freq='D')
df_base = pd.DataFrame({ 'date_local': rng}) 

In [34]:
df_base

,date_local
0,2010-01-01
1,2010-01-02
2,2010-01-03
3,2010-01-04
4,2010-01-05
...,...
4150,2021-05-13
4151,2021-05-14
4152,2021-05-15
4153,2021-05-16


In [36]:
unique_fields=merged_CO_Daily_filtered_v2.groupby(["state_code","county_code","state_name","county_name","parameter_name","units_of_measure"]).agg({"aqi_label":"max"}).reset_index()

unique_fields=unique_fields[["state_code","county_code","state_name","county_name","parameter_name","units_of_measure"]]
unique_fields.head()


,state_code,county_code,state_name,county_name,parameter_name,units_of_measure
0,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million
1,4,13,Arizona,Maricopa,Carbon monoxide,Parts per million
2,4,19,Arizona,Pima,Carbon monoxide,Parts per million
3,5,119,Arkansas,Pulaski,Carbon monoxide,Parts per million
4,6,1,California,Alameda,Carbon monoxide,Parts per million


In [37]:
df_base=df_base.merge(unique_fields, how='cross')

In [38]:
df_base

,date_local,state_code,county_code,state_name,county_name,parameter_name,units_of_measure
0,2010-01-01,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million
1,2010-01-01,4,13,Arizona,Maricopa,Carbon monoxide,Parts per million
2,2010-01-01,4,19,Arizona,Pima,Carbon monoxide,Parts per million
3,2010-01-01,5,119,Arkansas,Pulaski,Carbon monoxide,Parts per million
4,2010-01-01,6,1,California,Alameda,Carbon monoxide,Parts per million
...,...,...,...,...,...,...,...
415495,2021-05-17,49,49,Utah,Utah,Carbon monoxide,Parts per million
415496,2021-05-17,50,7,Vermont,Chittenden,Carbon monoxide,Parts per million
415497,2021-05-17,50,21,Vermont,Rutland,Carbon monoxide,Parts per million
415498,2021-05-17,51,13,Virginia,Arlington,Carbon monoxide,Parts per million


In [34]:
# df_base_missing=df_base.merge(merged_CO_Daily_filtered_v2,on=["state_code","county_code","state_name","county_name","parameter_name","units_of_measure"], how='left')

In [39]:
df_base_missing=df_base.merge(merged_CO_Daily_filtered_v2[["date_local","state_code","county_code","polutant_level"]],on=["date_local","state_code","county_code"], how='left')






In [42]:
 df_base_missing.head()

,date_local,state_code,county_code,state_name,county_name,parameter_name,units_of_measure,polutant_level
0,2010-01-01,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million,0.5
1,2010-01-01,4,13,Arizona,Maricopa,Carbon monoxide,Parts per million,3.1
2,2010-01-01,4,19,Arizona,Pima,Carbon monoxide,Parts per million,0.8
3,2010-01-01,5,119,Arkansas,Pulaski,Carbon monoxide,Parts per million,1.1
4,2010-01-01,6,1,California,Alameda,Carbon monoxide,Parts per million,0.7


In [43]:
df_base_missing=df_base_missing.sort_values(["state_code","county_code","date_local"])
df_base_missing

,date_local,state_code,county_code,state_name,county_name,parameter_name,units_of_measure,polutant_level
0,2010-01-01,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million,0.5
100,2010-01-02,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million,0.5
200,2010-01-03,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million,0.5
300,2010-01-04,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million,0.7
400,2010-01-05,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million,0.7
...,...,...,...,...,...,...,...,...
415099,2021-05-13,55,27,Wisconsin,Dodge,Carbon monoxide,Parts per million,NaN
415199,2021-05-14,55,27,Wisconsin,Dodge,Carbon monoxide,Parts per million,NaN
415299,2021-05-15,55,27,Wisconsin,Dodge,Carbon monoxide,Parts per million,NaN
415399,2021-05-16,55,27,Wisconsin,Dodge,Carbon monoxide,Parts per million,NaN


In [44]:
# # create series
# import numpy as np
# size = 2000
# x = np.linspace(-2, 5, size)
# y = pd.Series(np.sin(x))

# # deleting data segment
# y[10:30] = np.nan


# spline = y.interpolate(method='spline', order=1, s=0.)
# spline

In [45]:
# def interpolate_func(dataframe):
#     dataframe["polutant_level"]=dataframe["polutant_level"].interpolate(method='spline', order=3, s=0.)
#     return dataframe

In [46]:
final= pd.DataFrame(data=None, columns=df_base_missing.columns)
combo=list(zip(unique_fields["state_code"],unique_fields["county_code"]))
for state,county in combo:
    subset=df_base_missing[(df_base_missing.state_code==state)&(df_base_missing.county_code==county)]
    #print(subset)
    subset.set_index("date_local",inplace=True)
    subset["polutant_level2"]=subset["polutant_level"].interpolate(limit_direction="both",method='time', order=2, s=0.)
    subset.reset_index(inplace=True)
    #print(subset)
    final=final.append(subset)
    

<ipython-input-46-045b51f1db19>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["polutant_level2"]=subset["polutant_level"].interpolate(limit_direction="both",method='time', order=2, s=0.)
<ipython-input-46-045b51f1db19>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset["polutant_level2"]=subset["polutant_level"].interpolate(limit_direction="both",method='time', order=2, s=0.)
<ipython-input-46-045b51f1db19>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [47]:
final

,date_local,state_code,county_code,state_name,county_name,parameter_name,units_of_measure,polutant_level,polutant_level2
0,2010-01-01,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million,0.5,0.5
1,2010-01-02,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million,0.5,0.5
2,2010-01-03,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million,0.5,0.5
3,2010-01-04,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million,0.7,0.7
4,2010-01-05,1,73,Alabama,Jefferson,Carbon monoxide,Parts per million,0.7,0.7
...,...,...,...,...,...,...,...,...,...
4150,2021-05-13,55,27,Wisconsin,Dodge,Carbon monoxide,Parts per million,NaN,0.2
4151,2021-05-14,55,27,Wisconsin,Dodge,Carbon monoxide,Parts per million,NaN,0.2
4152,2021-05-15,55,27,Wisconsin,Dodge,Carbon monoxide,Parts per million,NaN,0.2
4153,2021-05-16,55,27,Wisconsin,Dodge,Carbon monoxide,Parts per million,NaN,0.2


In [48]:
final.to_csv("co_daily.csv")

In [108]:
# df_base_extract_final=df_base_missing.merge(final[["date_local","state_code","county_code","polutant_level2"]],on=["date_local","state_code","county_code"], how='left')

# df_base_extract_final.to_csv("co_daily.csv")
